In [1]:
import logging
from langdetect import detect, detect_langs, DetectorFactory
from googletrans import Translator
import speech_recognition as sr
import pyttsx3
import pycountry
import time

# Set detector factory seed for reproducibility
DetectorFactory.seed = 0

# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Translator and TTS Engine
translator = Translator()
tts_engine = pyttsx3.init()

def speak(text):
    """
    Use text-to-speech to speak the provided text.
    Displays a speaking symbol while speaking.
    """
    print("🎤 Speaking...")  # Symbol for speaking
    tts_engine.say(text)
    tts_engine.runAndWait()
    print("🛑 Speaking done.")  # Stop symbol when speaking is done

def get_language_name(language_code):
    """
    Get the language name from the ISO 639-1 language code using pycountry.
    """
    try:
        language = pycountry.languages.get(alpha_2=language_code)
        return language.name if language else "Unknown Language"
    except Exception as e:
        logging.error(f"Error getting language name: {e}")
        return "Unknown Language"

def detect_language(text):
    """
    Detect the language of the given text and provide confidence scores.
    Returns:
        language_code (str): ISO 639-1 language code
        confidence (float): Confidence score
    """
    try:
        possible_languages = detect_langs(text)
        best_guess = possible_languages[0]
        language_code = best_guess.lang
        confidence = best_guess.prob
        return language_code, confidence
    except Exception as e:
        logging.error(f"Error detecting language: {e}")
        return "Unknown", 0.0

def translate_to_english(text, src_language_code):
    """
    Translate the given text to English.
    Returns:
        translated_text (str): Text translated into English
    """
    try:
        translated_text = translator.translate(text, src=src_language_code, dest='en').text
        return translated_text
    except Exception as e:
        logging.error(f"Error translating text: {e}")
        return "Translation failed."

def listen():
    """
    Use the microphone to capture voice input and convert it to text.
    Displays a listening symbol while capturing audio.
    """
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("🎧 Listening...")  # Symbol for listening
        recognizer.adjust_for_ambient_noise(source)  # Automatically adjust for ambient noise

        print("Speak now:")
        try:
            audio = recognizer.listen(source, timeout=10, phrase_time_limit=5)  # Timeout and limit for better control
            text = recognizer.recognize_google(audio)  # Use Google's recognizer by default
            print(f"You said: {text}")
            print("🛑 Listening done.")  # Stop symbol when listening is done
            return text
        except sr.UnknownValueError:
            print("Sorry, I could not understand your speech. Please try again.")
            return None
        except sr.RequestError as e:
            print(f"Speech recognition error: {e}. Please check your internet connection.")
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

def main():
    """
    Main function to detect the language and translate text with voice support.
    """
    print("Language Detection and Translation Tool")
    print("Type 'exit' or say 'exit' to quit.\n")
    speak("Welcome to the Language Detection and Translation Tool. You can speak or type your text.")

    while True:
        print("Choose input method: (1) Speak (2) Type (3) Exit")
        speak("Please choose your input method: press one for speaking, two for typing, or three to exit.")
        method = input("Enter your choice (1/2/3): ").strip()

        if method == '1':
            text = listen()
            if text is None:
                continue
        elif method == '2':
            text = input("Enter the text: ").strip()
        elif method == '3':
            print("Exiting the tool. Goodbye!")
            speak("Exiting the tool. Goodbye!")
            break
        else:
            print("Invalid choice. Try again.")
            continue

        if text and (text.lower() == 'exit' or text.lower() == 'quit'):
            print("Exiting the tool. Goodbye!")
            speak("Exiting the tool. Goodbye!")
            break

        if len(text) < 5:
            print("Input text is too short. Please provide more content.")
            speak("The input text is too short. Please provide more content.")
            continue

        # Detect language
        language_code, confidence = detect_language(text)

        # Translate text to English
        if language_code != "Unknown":
            translated_text = translate_to_english(text, language_code)
            language_name = get_language_name(language_code)
        else:
            translated_text = "Unable to translate due to unknown language."
            language_name = "Unknown Language"

        # Print and speak results
        print(f"Detected Language: {language_name}")
        print(f"Confidence: {confidence:.2f}")
        print(f"Translated to English: {translated_text}\n")

        speak(f"The detected language is {language_name}. The translation to English is: {translated_text}")

if __name__ == "__main__":
    main()


Language Detection and Translation Tool
Type 'exit' or say 'exit' to quit.

🎤 Speaking...
🛑 Speaking done.
Choose input method: (1) Speak (2) Type (3) Exit
🎤 Speaking...
🛑 Speaking done.
Enter your choice (1/2/3): 2
Enter the text: This is a book
Detected Language: English
Confidence: 1.00
Translated to English: This is a book

🎤 Speaking...
🛑 Speaking done.
Choose input method: (1) Speak (2) Type (3) Exit
🎤 Speaking...
🛑 Speaking done.
Enter your choice (1/2/3): 2
Enter the text: यह एक किताब है
Detected Language: Hindi
Confidence: 1.00
Translated to English: This is a book

🎤 Speaking...
🛑 Speaking done.
Choose input method: (1) Speak (2) Type (3) Exit
🎤 Speaking...
🛑 Speaking done.
Enter your choice (1/2/3): 3
Exiting the tool. Goodbye!
🎤 Speaking...
🛑 Speaking done.
